# Part 3(b)

This notebook will highlight the process, result and insights obtained from using the embedding matrix built that includes OOV words with their contextual average as embeddings.

## Import Embedding Matrix

In [1]:
import json
from pathlib import Path

import numpy as np

embedding_path = Path("models/embedding_matrix_oov.npy")
index_from_word_path = Path("models/index_from_word_oov.json")

embedding_matrix = np.load(embedding_path)
with index_from_word_path.open() as f:
    index_from_word = json.load(f)

## Prepare Dataaset

(a) Import datasets

In [2]:
from utils.text import tokenize
from datasets import load_dataset

dataset = load_dataset("rotten_tomatoes")
train_dataset = tokenize(dataset["train"])
val_dataset = tokenize(dataset["validation"])
test_dataset = tokenize(dataset["test"])

/home/yuri/sc4002-nlp-sentiment-classification/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[nltk_data] Downloading package punkt to /home/yuri/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package treebank to /home/yuri/nltk_data...
[nltk_data]   Package treebank is already up-to-date!
[nltk_data] Downloading package punkt_tab to /home/yuri/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [3]:
from utils.text import token_to_index

train_dataset = token_to_index(dataset=train_dataset, index_from_word=index_from_word)
val_dataset = token_to_index(dataset=val_dataset, index_from_word=index_from_word)
test_dataset = token_to_index(dataset=test_dataset, index_from_word=index_from_word)

train_dataset

Dataset({
    features: ['text', 'label', 'tokens', 'original_len', 'indexes'],
    num_rows: 8530
})

In [4]:
train_dataset = train_dataset.select_columns(["label", "original_len", "indexes"])
val_dataset = val_dataset.select_columns(["label", "original_len", "indexes"])
test_dataset = test_dataset.select_columns(["label", "original_len", "indexes"])

In [5]:
train_dataset.set_format(type="torch")
val_dataset.set_format(type="torch")
test_dataset.set_format(type="torch")

## Train RNN

We use Optuna to perform heuristic search on optimal configuration when the embedding matrix with OOV words is updatable during training.

In [ ]:
import optuna
from utils.train import train_rnn_model_with_parameters

_N_TRIALS = 500
SEARCH_SPACE = {
    "batch_size": [32, 64, 128, 256, 512, 1024, 2048],
    "learning_rate": [1e-5, 1e-4, 1e-3, 1e-2, 1e-1],
    "optimizer_name": ["SGD", "Adagrad", "RMSprop", "Adam"],
    # RNN Model Parameters
    "hidden_dim": [256, 128, 64, 32],
    "num_layers": [1, 2, 4],
    "sentence_representation_type": ["last", "average", "max"],
}

def objective(trial):
    hidden_dim = trial.suggest_categorical("hidden_dim", SEARCH_SPACE["hidden_dim"])
    num_layers = trial.suggest_int("num_layers", min(SEARCH_SPACE["num_layers"]), max(SEARCH_SPACE["num_layers"]))
    optimizer_name = trial.suggest_categorical("optimizer_name", SEARCH_SPACE["optimizer_name"])
    batch_size = trial.suggest_categorical("batch_size", SEARCH_SPACE["batch_size"])
    learning_rate = trial.suggest_categorical("learning_rate", SEARCH_SPACE["learning_rate"])
    sentence_representation_type = trial.suggest_categorical("sentence_representation_type", SEARCH_SPACE["sentence_representation_type"])
    
    log_message = f"---------- batch_size_{batch_size}; lr_{learning_rate}; optimizer_{optimizer_name}; hidden_dim_{hidden_dim}; num_layers_{num_layers}; sentence_representation_{sentence_representation_type} ----------"
    print(log_message)

    val_acc = train_rnn_model_with_parameters(
        embedding_matrix=embedding_matrix,
        train_dataset=train_dataset,
        val_dataset=val_dataset,
        batch_size=batch_size,
        learning_rate=learning_rate,
        optimizer_name=optimizer_name,
        hidden_dim=hidden_dim,
        num_layers=num_layers,
        sentence_representation_type=sentence_representation_type,
        show_progress=True,
        freeze_embedding=False,
        log_dir="rnn_trainable_embeddings_and_contextual_oov"
    )
    
    return val_acc

# Set up the Optuna study
study = optuna.create_study(direction="maximize") 
study.optimize(objective, n_trials=_N_TRIALS)

best_params = study.best_params

## Train Result Analysis

(a) Load result from all trials

In [6]:
from utils.analytics import load_tensorboard_logs

train_results_df = load_tensorboard_logs(
    log_dir="tb_logs/rnn_trainable_embeddings_and_contextual_oov"
)

train_results_df = train_results_df.sort_values(
    by=["val_acc"], ascending=False
).reset_index(drop=True)
train_results_df.head(20)

,val_acc,train_acc,batch_size,hidden_dim,learning_rate,optimizer_name,num_layers,sentence_representation_type,freeze,epoch,train_loss,val_loss,filename
0,0.778612,0.997041,2048,64,0.0100,Adam,1,max,False,9.0,0.006933,0.486345,events.out.tfevents.1730824964.yuriarch.255181...
1,0.773921,0.985282,2048,128,0.0100,Adagrad,1,max,False,9.0,0.081584,0.477558,events.out.tfevents.1730824258.yuriarch.255181...
2,0.771107,0.956721,2048,128,0.0100,Adagrad,1,average,False,9.0,0.149831,0.526909,events.out.tfevents.1730823198.yuriarch.255181...
3,0.768293,1.000000,2048,32,0.0100,Adam,1,max,False,9.0,0.004599,0.492995,events.out.tfevents.1730825686.yuriarch.255181...
4,0.768293,0.872232,2048,256,0.0001,Adam,1,max,False,25.0,0.302687,0.498160,events.out.tfevents.1730826915.yuriarch.255181...
5,0.768293,0.925914,2048,32,0.0001,Adam,1,max,False,75.0,0.326016,0.529641,events.out.tfevents.1730828595.yuriarch.255181...
6,0.767355,0.970628,2048,32,0.0010,RMSprop,1,max,False,10.0,0.128261,0.486081,events.out.tfevents.1730826888.yuriarch.255181...
7,0.766417,0.901121,2048,128,0.0100,Adagrad,3,max,False,11.0,0.211342,0.536393,events.out.tfevents.1730824603.yuriarch.255181...
8,0.765478,0.952858,2048,256,0.0010,RMSprop,1,average,False,10.0,0.143535,0.502480,events.out.tfevents.1730822963.yuriarch.255181...
9,0.764540,0.985034,2048,64,0.0100,Adagrad,3,max,False,9.0,0.079645,0.501845,events.out.tfevents.1730827890.yuriarch.255181...


(b) Configuration for best trial result

In [7]:
best_rnn_model_configuration = train_results_df.head(1)
best_rnn_model_configuration

,val_acc,train_acc,batch_size,hidden_dim,learning_rate,optimizer_name,num_layers,sentence_representation_type,freeze,epoch,train_loss,val_loss,filename
0,0.778612,0.997041,2048,64,0.01,Adam,1,max,False,9.0,0.006933,0.486345,events.out.tfevents.1730824964.yuriarch.255181...


## Performance on Test Dataset

In [ ]:
from utils.analytics import test_top_n_models
from models.RNN import RNNClassifier

test_results_df = test_top_n_models(train_results_df, RNNClassifier, test_dataset, n=10)

In [12]:
test_results_df

,test_acc,test_loss,val_acc,train_acc,batch_size,hidden_dim,learning_rate,optimizer_name,num_layers,sentence_representation_type,freeze,epoch,train_loss,val_loss,filename
0,0.744841,1.187047,0.778612,0.997041,2048,64,0.0100,Adam,1,max,False,9.0,0.006933,0.486345,events.out.tfevents.1730824964.yuriarch.255181...
1,0.769231,0.519958,0.773921,0.985282,2048,128,0.0100,Adagrad,1,max,False,9.0,0.081584,0.477558,events.out.tfevents.1730824258.yuriarch.255181...
2,0.773921,1.132022,0.771107,0.956721,2048,128,0.0100,Adagrad,1,average,False,9.0,0.149831,0.526909,events.out.tfevents.1730823198.yuriarch.255181...
3,0.732645,1.169337,0.768293,1.000000,2048,32,0.0100,Adam,1,max,False,9.0,0.004599,0.492995,events.out.tfevents.1730825686.yuriarch.255181...
4,0.780488,0.487552,0.768293,0.872232,2048,256,0.0001,Adam,1,max,False,25.0,0.302687,0.498160,events.out.tfevents.1730826915.yuriarch.255181...
5,0.772983,0.524823,0.768293,0.925914,2048,32,0.0001,Adam,1,max,False,75.0,0.326016,0.529641,events.out.tfevents.1730828595.yuriarch.255181...
6,0.792683,0.45683,0.767355,0.970628,2048,32,0.0010,RMSprop,1,max,False,10.0,0.128261,0.486081,events.out.tfevents.1730826888.yuriarch.255181...
7,0.762664,0.527458,0.766417,0.901121,2048,128,0.0100,Adagrad,3,max,False,11.0,0.211342,0.536393,events.out.tfevents.1730824603.yuriarch.255181...
8,0.77955,1.087536,0.765478,0.952858,2048,256,0.0010,RMSprop,1,average,False,10.0,0.143535,0.502480,events.out.tfevents.1730822963.yuriarch.255181...
9,0.767355,0.519988,0.764540,0.985034,2048,64,0.0100,Adagrad,3,max,False,9.0,0.079645,0.501845,events.out.tfevents.1730827890.yuriarch.255181...
